In [1]:
import numpy as np
import pandas as pd
import cv2
import os
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from sklearn.model_selection import train_test_split
fileRoot = "D:\\ICLR-CropDisease\\dataset\\"

In [2]:
gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
print(gpus)

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8273218242456770809
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 9210949796
locality {
  bus_id: 1
  links {
  }
}
incarnation: 17353790831532784825
physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [3]:
img_shape_x = 64
img_shape_y = 64

In [4]:
# Load data and preprocess
def loadData():
    print("loadData called")
    
    data_img = []
    data_label = []

    for file in os.listdir(fileRoot + "train\\healthy_wheat\\"):
        img = cv2.imread(fileRoot + "train\\healthy_wheat\\" + file)
        res = cv2.resize(img, dsize=(128, 128))
        #print(res.shape)
        res = crop_center(res, img_shape_x, img_shape_y)
        #print(res.shape)
        data_img.append(res)
        data_label.append(0)

    for file in os.listdir(fileRoot + "train\\leaf_rust\\"):
        img = cv2.imread(fileRoot + "train\\leaf_rust\\" + file)
        if img is None:
            print(file)
            continue
        res = cv2.resize(img, dsize=(128, 128))
        res = crop_center(res, 64, 64)
        data_img.append(res)
        data_label.append(1)

    for file in os.listdir(fileRoot + "train\\stem_rust\\"):
        img = cv2.imread(fileRoot + "train\\stem_rust\\" + file)
        res = cv2.resize(img, dsize=(128, 128))
        res = crop_center(res, 64, 64)
        data_img.append(res)
        data_label.append(2)

    for i in range(len(data_img)):
        data_img[i] = data_img[i] / 255
    data_img = np.array(data_img)
    return data_img, data_label

def crop_center(img, x_dim, y_dim):
    #print("crop_center called")
    y,x,_ = img.shape
    startx = x//2-(x_dim//2)
    starty = y//2-(y_dim//2)    
    return img[starty:starty+y_dim,startx:startx+x_dim,]

def trainTestSplit(data_img,data_label):
    print("trainTestSplit called")
    X_train, X_test, y_train, y_test = train_test_split(data_img,data_label,test_size = 0.3)
    return X_train, X_test, y_train, y_test



In [ ]:
data_img,data_lable = loadData()
X_train, X_test, y_train, y_test = trainTestSplit(data_img,data_lable)

loadData called


In [ ]:
#Create CNN model
# Current : 3 conv layers, 2 pooling, 1 flatten, 2 dense.
def trainModel(X_train, X_test, y_train, y_test):
    print ("trainModel called")

    y_train =np.array(y_train)
    y_test =np.array(y_test)
    model = models.Sequential()
    model.add(layers.Conv2D(128, (3, 3), activation='relu', input_shape=(img_shape_x, img_shape_y, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(3))

    model.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])

    # Train
    history = model.fit(X_train, y_train, epochs=10)

    # test
    test_loss, test_acc = model.evaluate(X_test,y_test, verbose=2)
    print(test_acc)

In [ ]:
trainModel(X_train, X_test, y_train, y_test)